#  Downloading Audio
## Anna Bernbaum
## April 2019

Problems with using the Google AudioSet TFRecords:
- Lack of understanding what '128 - dimension audio features extracted at 1 Hz' actually means
- Difference between frame level features and video level features is unclear
- Embedding? Can you un-embed something?

New tactic:
- Download the actual audio clips for all speech and cough samples from youtube
- Create a spectrogram (or mel spectrogram) for each 
- Train the model on these images

In [1]:
import pandas as pd
import youtube_dl
from pydub import AudioSegment
from pydub.utils import which
from pydub.utils import mediainfo
import librosa as librosa
import os
from random import shuffle
import csv

import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = [20, 5]

# Get all desired YouTubeIDs

Let's find the index and code associated with 'Cough':

In [2]:
cough_class_label_index = !grep Cough AudioSet/class_labels_indices.csv
print(cough_class_label_index)

speech_class_label_index = !grep Speech AudioSet/class_labels_indices.csv
print(speech_class_label_index)


['47,/m/01b_21,"Cough"']
['0,/m/09x0r,"Speech"', '7,/m/0brhx,"Speech synthesizer"']


Retreiving the class label:

In [3]:
print("Cough:", cough_class_label_index[0].split(",")[1])
print("Speech:", speech_class_label_index[0].split(",")[1])

Cough: /m/01b_21
Speech: /m/09x0r


Finding all samples with the selected label. The header of this table is printed.

In [4]:
# coughs = !grep /m/01b_21 AudioSet/balanced_train_segments.csv |head  # ID manually inserted
# print(type(coughs))
# print(coughs)

# Maybe select speech clips with only a speech tag
# ignore clips with speech and cough tag

coughs_bal = !grep /m/01b_21 AudioSet/balanced_train_segments.csv | cut -c -11
coughs_bal_starts = !grep /m/01b_21 AudioSet/balanced_train_segments.csv | cut -d ',' -f 2
print("Cough samples in Balanced Train:", len(coughs_bal))

speech_bal = !grep /m/09x0r AudioSet/balanced_train_segments.csv | cut -c -11
speech_bal_starts = !grep /m/09x0r AudioSet/balanced_train_segments.csv | cut -d ',' -f 2
print("Speech samples in Balanced Train:", len(speech_bal))


coughs_eval = !grep /m/01b_21 AudioSet/eval_segments.csv | cut -c -11
coughs_eval_starts = !grep /m/01b_21 AudioSet/eval_segments.csv | cut -d ',' -f 2
print("\nCough samples in Evaluation:", len(coughs_eval))

speech_eval = !grep /m/09x0r AudioSet/eval_segments.csv | cut -c -11
speech_eval_starts = !grep /m/09x0r AudioSet/eval_segments.csv | cut -d ',' -f 2
print("Speech samples in Evaluation:", len(speech_eval))


coughs_unbal = !grep /m/01b_21 AudioSet/unbalanced_train_segments.csv | cut -c -11
coughs_unbal_starts = !grep /m/01b_21 AudioSet/unbalanced_train_segments.csv | cut -d ',' -f 2
print("\nCough samples in Unbalanced Train:", len(coughs_unbal))

speech_unbal = !grep /m/09x0r AudioSet/unbalanced_train_segments.csv | cut -c -11
speech_unbal_starts = !grep /m/09x0r AudioSet/unbalanced_train_segments.csv | cut -d ',' -f 2
print("Speech samples in Unbalanced Train:", len(speech_unbal))
      
print("\nTotal Number of Cough Clips:", (len(coughs_bal)+len(coughs_eval)+len(coughs_unbal)))
print("\nTotal Number of Speech Clips:", (len(speech_bal)+len(speech_eval)+len(speech_unbal)))

Cough samples in Balanced Train: 60
Speech samples in Balanced Train: 5735

Cough samples in Evaluation: 60
Speech samples in Evaluation: 5324

Cough samples in Unbalanced Train: 751
Speech samples in Unbalanced Train: 999421

Total Number of Cough Clips: 871

Total Number of Speech Clips: 1010480


Let's create our own Train : Evaluation : Validation split to maximise the number of clips used. This will be done in a separate file, but here we combine all available clips.

In [5]:
# combine the lists of youtube ids into one master list
all_coughs = (coughs_bal + coughs_eval + coughs_unbal)
all_speech = (speech_bal + speech_eval + speech_unbal)

all_coughs_starts = (coughs_bal_starts + coughs_eval_starts + coughs_unbal_starts)
all_speech_starts = (speech_bal_starts + speech_eval_starts + speech_unbal_starts)


# create pandas dataframe of YouTubeIDs
speech_df = pd.DataFrame()  # create empty dataframe
cough_df = pd.DataFrame()
speech_df['Speech YouTubeIDs'] = pd.Series(all_speech)
cough_df['Coughs YouTubeIDs'] = pd.Series(all_coughs)
speech_df['Speech Starts'] = pd.Series(all_speech_starts)
cough_df['Coughs Starts'] = pd.Series(all_coughs_starts)

# Shuffle the dataframes
speech_df = speech_df.sample(frac=1).reset_index(drop=True)
cough_df = cough_df.sample(frac=1).reset_index(drop=True)

# Trim Speech Df
speech_df = speech_df[:871]

print(speech_df)
print(cough_df)

    Speech YouTubeIDs Speech Starts
0         Uh-m7J2t1gQ        30.000
1         4XCquOXip9E        30.000
2         9rVz4d19TKs       210.000
3         2Se5wE9k_R8       300.000
4         R7uT1uJLaFA       430.000
5         HZ89D3rNjkU        30.000
6         At1sT4uNu_E        30.000
7         ep2jwIMGcXU        20.000
8         cnOtOIVLL2I        30.000
9         HyEZ5MKbRFs       100.000
10        yFI4CGyuSgs         0.000
11        AGV6S7WYA98       120.000
12        CJqwWut9-qA        14.000
13        oSQF3lATOkA        30.000
14        7VJXBELY_UQ       180.000
15        DkA1ZnkI5NE       150.000
16        -W24HpMxqus        30.000
17        SWSymPKMWP8        30.000
18        -Igtk68gRas        30.000
19        FRlDIdNoxEA        30.000
20        rL8w7geUdaI        30.000
21        O4mLCMBg50I        30.000
22        qzT1A7a9W0E        30.000
23        FmUKiCzLXjw        30.000
24        _2bhs5VUtQQ        30.000
25        sb_Fp0xoRzY       210.000
26        Z9YZIYBMxEU       

# Find Associated YouTube clip for each YouTubeID

https://stackoverflow.com/questions/27473526/download-only-audio-from-youtube-video-using-youtube-dl-in-python-script

https://stackoverflow.com/questions/28423501/download-part-of-the-youtube-video-using-python


## Getting all Speech Clips

In [10]:
speech_test_df = speech_df[:300]

speech_fail_log = []

for i in range(len(speech_test_df.index)):
    title = speech_test_df['Speech YouTubeIDs'][i] + '.wav'  # filename
    url = "http://www.youtube.com/watch?v=" + title  # URL

    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': 'Untrimmed_AudioSet_WAV_files/' + title,
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'wav',  # WAV file
            'preferredquality': '192',
        }],
    }
    
    try:
        with youtube_dl.YoutubeDL(ydl_opts) as ydl:
            ydl.download([url])

        # Find the associated start_time
        start_time = float(speech_test_df['Speech Starts'][i])

        # Trim the clip
        y, sr = librosa.load('Untrimmed_AudioSet_WAV_files/'+ title, offset=start_time, duration=10.0) # trim a 10 second segment from start_time    

    #     # plot the clip - used for debugging
    #     plt.title(title)
    #     plt.xlabel('sample number')
    #     plt.ylabel('Amplitude')
    #     plt.plot(y)
    #     plt.show()

        # write a wav file of the trimmed clip
        filepath = 'Audioset_WAV_files/Speech/' + title
        librosa.output.write_wav(filepath, y, sr, norm=True)
        librosa.output.write_wav(filepath, y, sr, norm=True)

        
    # if the clip is unavailable
    except: 
        speech_fail_log.append(speech_test_df['Speech YouTubeIDs'][i])

        
print(speech_fail_log)

[youtube] Uh-m7J2t1gQ: Downloading webpage
[youtube] Uh-m7J2t1gQ: Downloading video info webpage
[download] Destination: Untrimmed_AudioSet_WAV_files/Uh-m7J2t1gQ.wav
[download] 100% of 1.99MiB in 00:0056MiB/s ETA 00:002
[ffmpeg] Post-process file Untrimmed_AudioSet_WAV_files/Uh-m7J2t1gQ.wav exists, skipping
[youtube] 4XCquOXip9E: Downloading webpage
[youtube] 4XCquOXip9E: Downloading video info webpage
[download] Destination: Untrimmed_AudioSet_WAV_files/4XCquOXip9E.wav
[download] 100% of 8.63MiB in 00:0161MiB/s ETA 00:0008
[ffmpeg] Post-process file Untrimmed_AudioSet_WAV_files/4XCquOXip9E.wav exists, skipping
[youtube] 9rVz4d19TKs: Downloading webpage
[youtube] 9rVz4d19TKs: Downloading video info webpage
[download] Destination: Untrimmed_AudioSet_WAV_files/9rVz4d19TKs.wav
[download] 100% of 5.45MiB in 00:0015MiB/s ETA 00:009
[ffmpeg] Correcting container in "Untrimmed_AudioSet_WAV_files/9rVz4d19TKs.wav"
[ffmpeg] Post-process file Untrimmed_AudioSet_WAV_files/9rVz4d19TKs.wav exists, s

ERROR: This video is unavailable.


[youtube] HyEZ5MKbRFs: Downloading webpage
[youtube] HyEZ5MKbRFs: Downloading video info webpage


ERROR: This video is unavailable.


[youtube] yFI4CGyuSgs: Downloading webpage
[youtube] yFI4CGyuSgs: Downloading video info webpage
[download] Destination: Untrimmed_AudioSet_WAV_files/yFI4CGyuSgs.wav
[download] 100% of 40.31KiB in 00:0010MiB/s ETA 00:000
[ffmpeg] Correcting container in "Untrimmed_AudioSet_WAV_files/yFI4CGyuSgs.wav"
[ffmpeg] Post-process file Untrimmed_AudioSet_WAV_files/yFI4CGyuSgs.wav exists, skipping
[youtube] AGV6S7WYA98: Downloading webpage
[youtube] AGV6S7WYA98: Downloading video info webpage
[download] Destination: Untrimmed_AudioSet_WAV_files/AGV6S7WYA98.wav
[download] 100% of 14.67MiB in 00:0108MiB/s ETA 00:005
[ffmpeg] Correcting container in "Untrimmed_AudioSet_WAV_files/AGV6S7WYA98.wav"
[ffmpeg] Post-process file Untrimmed_AudioSet_WAV_files/AGV6S7WYA98.wav exists, skipping
[youtube] CJqwWut9-qA: Downloading webpage
[youtube] CJqwWut9-qA: Downloading video info webpage
[download] Destination: Untrimmed_AudioSet_WAV_files/CJqwWut9-qA.wav
[download] 100% of 389.20KiB in 00:0081MiB/s ETA 00:00

...

[youtube] m2GnrbxJdXg: Downloading webpage
[youtube] m2GnrbxJdXg: Downloading video info webpage
[download] Destination: Untrimmed_AudioSet_WAV_files/m2GnrbxJdXg.wav
[download] 100% of 16.28MiB in 00:0034MiB/s ETA 00:008
[ffmpeg] Post-process file Untrimmed_AudioSet_WAV_files/m2GnrbxJdXg.wav exists, skipping
[youtube] 3Mp_j3KZyXw: Downloading webpage
[youtube] 3Mp_j3KZyXw: Downloading video info webpage
[download] Destination: Untrimmed_AudioSet_WAV_files/3Mp_j3KZyXw.wav
[download] 100% of 7.73MiB in 00:0138MiB/s ETA 00:003
[ffmpeg] Post-process file Untrimmed_AudioSet_WAV_files/3Mp_j3KZyXw.wav exists, skipping
[youtube] 7xgOMbIIH4k: Downloading webpage
[youtube] 7xgOMbIIH4k: Downloading video info webpage
[youtube] 7xgOMbIIH4k: Downloading MPD manifest
[dashsegments] Total fragments: 17
[download] Destination: Untrimmed_AudioSet_WAV_files/7xgOMbIIH4k.wav
[download] 100% of 1.72MiB in 00:03.87MiB/s ETA 00:000524
[ffmpeg] Correcting container in "Untrimmed_AudioSet_WAV_files/7xgOMbIIH4k

ERROR: This video is unavailable.


[youtube] lUf8KCNHh0M: Downloading webpage
[youtube] lUf8KCNHh0M: Downloading video info webpage
[download] Destination: Untrimmed_AudioSet_WAV_files/lUf8KCNHh0M.wav
[download] 100% of 935.47KiB in 00:0019MiB/s ETA 00:001
[ffmpeg] Post-process file Untrimmed_AudioSet_WAV_files/lUf8KCNHh0M.wav exists, skipping
[youtube] sSV_xCETDl8: Downloading webpage
[youtube] sSV_xCETDl8: Downloading video info webpage
[download] Destination: Untrimmed_AudioSet_WAV_files/sSV_xCETDl8.wav
[download] 100% of 8.86MiB in 00:0140MiB/s ETA 00:005
[ffmpeg] Post-process file Untrimmed_AudioSet_WAV_files/sSV_xCETDl8.wav exists, skipping
[youtube] aKevq_vcE0M: Downloading webpage
[youtube] aKevq_vcE0M: Downloading video info webpage
[download] Destination: Untrimmed_AudioSet_WAV_files/aKevq_vcE0M.wav
[download] 100% of 822.88KiB in 00:0009MiB/s ETA 00:001
[ffmpeg] Correcting container in "Untrimmed_AudioSet_WAV_files/aKevq_vcE0M.wav"
[ffmpeg] Post-process file Untrimmed_AudioSet_WAV_files/aKevq_vcE0M.wav exists

ERROR: This video is unavailable.


[youtube] dFh6WPSGLys: Downloading webpage
[youtube] dFh6WPSGLys: Downloading video info webpage
[download] Destination: Untrimmed_AudioSet_WAV_files/dFh6WPSGLys.wav
[download] 100% of 4.26MiB in 00:0002MiB/s ETA 00:006
[ffmpeg] Post-process file Untrimmed_AudioSet_WAV_files/dFh6WPSGLys.wav exists, skipping
[youtube] x4-YsPb4P-E: Downloading webpage
[youtube] x4-YsPb4P-E: Downloading video info webpage
[download] Destination: Untrimmed_AudioSet_WAV_files/x4-YsPb4P-E.wav
[download] 100% of 4.12MiB in 00:0032MiB/s ETA 00:007
[ffmpeg] Post-process file Untrimmed_AudioSet_WAV_files/x4-YsPb4P-E.wav exists, skipping
[youtube] Hi9alQXfPxo: Downloading webpage
[youtube] Hi9alQXfPxo: Downloading video info webpage
[download] Destination: Untrimmed_AudioSet_WAV_files/Hi9alQXfPxo.wav
[download] 100% of 912.61KiB in 00:0003MiB/s ETA 00:000
[ffmpeg] Correcting container in "Untrimmed_AudioSet_WAV_files/Hi9alQXfPxo.wav"
[ffmpeg] Post-process file Untrimmed_AudioSet_WAV_files/Hi9alQXfPxo.wav exists, 

## Getting all cough clips

In [11]:
coughs_test_df = cough_df[:300]

coughs_fail_log = []

for i in range(len(coughs_test_df.index)):
    title = coughs_test_df['Coughs YouTubeIDs'][i] + '.wav'  # filename
    url = "http://www.youtube.com/watch?v=" + title  # URL

    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': 'Untrimmed_AudioSet_WAV_files/' + title,
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'wav',  # WAV file
            'preferredquality': '192',
        }],
    }
    
    # if clip is unavailable
    # if clip is longer than X skip
    try:
        with youtube_dl.YoutubeDL(ydl_opts) as ydl:
            ydl.download([url])
        # Find the associated start_time
        start_time = float(coughs_test_df['Coughs Starts'][i])


        # Trim the clip
        y, sr = librosa.load('Untrimmed_AudioSet_WAV_files/' + title, offset=start_time, duration=10.0) # trim a 10 second segment from start_time    

    #     # plot the clip - used for debugging
    #     plt.title(title)
    #     plt.xlabel('sample number')
    #     plt.ylabel('Amplitude')
    #     plt.plot(y)
    #     plt.show()

        # write a wav file of the trimmed clip
        filepath = 'Audioset_WAV_files/Coughs/' + title
        librosa.output.write_wav(filepath, y, sr, norm=True)
        librosa.output.write_wav(filepath, y, sr, norm=True)

    # if clip is unavailable
    except: 
        coughs_fail_log.append(coughs_test_df['Coughs YouTubeIDs'][i])
        # create an error log

print(coughs_fail_log)

[youtube] 8ieJbzu7ql8: Downloading webpage
[youtube] 8ieJbzu7ql8: Downloading video info webpage
[youtube] 8ieJbzu7ql8: Downloading MPD manifest
[dashsegments] Total fragments: 6
[download] Destination: Untrimmed_AudioSet_WAV_files/8ieJbzu7ql8.wav
[download] 100% of 677.33KiB in 00:02.63MiB/s ETA 00:000
[ffmpeg] Correcting container in "Untrimmed_AudioSet_WAV_files/8ieJbzu7ql8.wav"
[ffmpeg] Post-process file Untrimmed_AudioSet_WAV_files/8ieJbzu7ql8.wav exists, skipping
[youtube] LDCHOGG9jqo: Downloading webpage
[youtube] LDCHOGG9jqo: Downloading video info webpage
[download] Destination: Untrimmed_AudioSet_WAV_files/LDCHOGG9jqo.wav
[download] 100% of 918.60KiB in 00:0047MiB/s ETA 00:001
[ffmpeg] Correcting container in "Untrimmed_AudioSet_WAV_files/LDCHOGG9jqo.wav"
[ffmpeg] Post-process file Untrimmed_AudioSet_WAV_files/LDCHOGG9jqo.wav exists, skipping
[youtube] f1zWL7xBcCo: Downloading webpage
[youtube] f1zWL7xBcCo: Downloading video info webpage
[download] Destination: Untrimmed_Audi

ERROR: This video is unavailable.


[youtube] P4Vp5cMKjL0: Downloading webpage
[youtube] P4Vp5cMKjL0: Downloading video info webpage
[download] Destination: Untrimmed_AudioSet_WAV_files/P4Vp5cMKjL0.wav
[download] 100% of 1.57MiB in 00:0012MiB/s ETA 00:002
[ffmpeg] Correcting container in "Untrimmed_AudioSet_WAV_files/P4Vp5cMKjL0.wav"
[ffmpeg] Post-process file Untrimmed_AudioSet_WAV_files/P4Vp5cMKjL0.wav exists, skipping
[youtube] v2nem-VUHNE: Downloading webpage
[youtube] v2nem-VUHNE: Downloading video info webpage
[download] Destination: Untrimmed_AudioSet_WAV_files/v2nem-VUHNE.wav
[download] 100% of 1.78MiB in 00:0098MiB/s ETA 00:002
[ffmpeg] Correcting container in "Untrimmed_AudioSet_WAV_files/v2nem-VUHNE.wav"
[ffmpeg] Post-process file Untrimmed_AudioSet_WAV_files/v2nem-VUHNE.wav exists, skipping
[youtube] 3fz-0wek44c: Downloading webpage
[youtube] 3fz-0wek44c: Downloading video info webpage
[download] Destination: Untrimmed_AudioSet_WAV_files/3fz-0wek44c.wav
[download] 100% of 136.14KiB in 00:0041MiB/s ETA 00:000


ERROR: This video is unavailable.


[youtube] NTwtl5HXYTE: Downloading webpage
[youtube] NTwtl5HXYTE: Downloading video info webpage
[download] Destination: Untrimmed_AudioSet_WAV_files/NTwtl5HXYTE.wav
[download] 100% of 861.18KiB in 00:0000MiB/s ETA 00:001
[ffmpeg] Post-process file Untrimmed_AudioSet_WAV_files/NTwtl5HXYTE.wav exists, skipping
[youtube] XJ0AWvDcnLM: Downloading webpage
[youtube] XJ0AWvDcnLM: Downloading video info webpage
[download] Destination: Untrimmed_AudioSet_WAV_files/XJ0AWvDcnLM.wav
[download] 100% of 729.77KiB in 00:0009MiB/s ETA 00:001
[ffmpeg] Correcting container in "Untrimmed_AudioSet_WAV_files/XJ0AWvDcnLM.wav"
[ffmpeg] Post-process file Untrimmed_AudioSet_WAV_files/XJ0AWvDcnLM.wav exists, skipping
[youtube] 6nTcsNoIGDw: Downloading webpage
[youtube] 6nTcsNoIGDw: Downloading video info webpage
[download] Destination: Untrimmed_AudioSet_WAV_files/6nTcsNoIGDw.wav
[download] 100% of 1.33MiB in 00:0079MiB/s ETA 00:002
[ffmpeg] Correcting container in "Untrimmed_AudioSet_WAV_files/6nTcsNoIGDw.wav

...

[youtube] 3gFShyvw8iU: Downloading webpage
[youtube] 3gFShyvw8iU: Downloading video info webpage
[download] Destination: Untrimmed_AudioSet_WAV_files/3gFShyvw8iU.wav
[download] 100% of 1.33MiB in 00:0091MiB/s ETA 00:001
[ffmpeg] Correcting container in "Untrimmed_AudioSet_WAV_files/3gFShyvw8iU.wav"
[ffmpeg] Post-process file Untrimmed_AudioSet_WAV_files/3gFShyvw8iU.wav exists, skipping
[youtube] cK_hw1SsdEM: Downloading webpage
[youtube] cK_hw1SsdEM: Downloading video info webpage
[download] Destination: Untrimmed_AudioSet_WAV_files/cK_hw1SsdEM.wav
[download] 100% of 1021.89KiB in 00:0081MiB/s ETA 00:001
[ffmpeg] Post-process file Untrimmed_AudioSet_WAV_files/cK_hw1SsdEM.wav exists, skipping
[youtube] aeWiKB1oEKo: Downloading webpage
[youtube] aeWiKB1oEKo: Downloading video info webpage
[download] Destination: Untrimmed_AudioSet_WAV_files/aeWiKB1oEKo.wav
[download] 100% of 327.51KiB in 00:0065MiB/s ETA 00:000
[ffmpeg] Post-process file Untrimmed_AudioSet_WAV_files/aeWiKB1oEKo.wav exist

ERROR: This video is unavailable.


[youtube] A6Ib82-X4Es: Downloading webpage
[youtube] A6Ib82-X4Es: Downloading video info webpage
[download] Destination: Untrimmed_AudioSet_WAV_files/A6Ib82-X4Es.wav
[download] 100% of 357.17KiB in 00:0002MiB/s ETA 00:000
[ffmpeg] Correcting container in "Untrimmed_AudioSet_WAV_files/A6Ib82-X4Es.wav"
[ffmpeg] Post-process file Untrimmed_AudioSet_WAV_files/A6Ib82-X4Es.wav exists, skipping
[youtube] pMse0rtj68g: Downloading webpage
[youtube] pMse0rtj68g: Downloading video info webpage
[youtube] pMse0rtj68g: Downloading MPD manifest
[dashsegments] Total fragments: 9
[download] Destination: Untrimmed_AudioSet_WAV_files/pMse0rtj68g.wav
[download] 100% of 552.03KiB in 00:00.97MiB/s ETA 00:0001
[ffmpeg] Correcting container in "Untrimmed_AudioSet_WAV_files/pMse0rtj68g.wav"
[ffmpeg] Post-process file Untrimmed_AudioSet_WAV_files/pMse0rtj68g.wav exists, skipping
[youtube] dWkml_CdR5w: Downloading webpage
[youtube] dWkml_CdR5w: Downloading video info webpage
[download] Destination: Untrimmed_Aud

ERROR: This video is unavailable.


[youtube] w4enmhllLkQ: Downloading webpage
[youtube] w4enmhllLkQ: Downloading video info webpage
[download] Destination: Untrimmed_AudioSet_WAV_files/w4enmhllLkQ.wav
[download] 100% of 464.32KiB in 00:0087MiB/s ETA 00:000
[ffmpeg] Correcting container in "Untrimmed_AudioSet_WAV_files/w4enmhllLkQ.wav"
[ffmpeg] Post-process file Untrimmed_AudioSet_WAV_files/w4enmhllLkQ.wav exists, skipping
['3KeyvLYtAWo', 'FRcvVvwMu_w', '3fJcwTKow5c', 'bGib_yHz3LQ', 'lvtn_PxRQm4', 'CsNTOw4LTBw', 'EX0ZF-jNWIg', '9AGw1Zz5Cvw', 'ho-ixLn-IzQ', 'NC30VSM-raQ', 'DuW_UxyOD-4', 'j6gaw7Nasnc', 'HMmbLN0NoYo', '4R36RB74SUQ', '6cCmLjEFkzs', 'vUV6wLqozYg', 'CJdOkPES7dA', 'eaZS0BWwQIU', 'AeWINP1ELkg', '8If9muUC-ZI', 'd6ceewlwkPU', 'gJyN6f0Pp28', 'UhD21M8FjAE', 'nK04u0NuenA', 'Eu8zDSaMWrA', 'Vp7pQzTCuXs', 'HJg4jZNv1pc', '39w1zX48tEg', 'n_fS-qHvLp8', 'eUUoaCAZ-DI', 'nI11IukPzUY', 'vIBtwK5mH5Q', 'YR5rkWyzUkQ', 'O8IBqEm7ilo', 'KTXYBXRVzxA']


## Check number of clips after downloading
Some clips were lost due to failed downloads. The two datagroups need to be evened out.

In [12]:
# Check number of clips in each folder
coughs_count=0
for files in os.listdir("Audioset_WAV_files/Coughs/"):
    if files.endswith('.wav'):
        coughs_count+=1
print("Unedited number of cough clips:", coughs_count)


speech_count=0
for files in os.listdir("Audioset_WAV_files/Speech/"):
    if files.endswith('.wav'):
        speech_count+=1
print("Unedited number of speech clips:", speech_count)

# Remove additional clips to even out the size
values = {"Coughs": coughs_count, "Speech": speech_count}
largest = max(values, key=values.get)  # find which group has more clips
difference = (abs(coughs_count - speech_count))  # how many clips should be removed

# find names of clips to be removed
f = []
for (dirpath, dirnames, filenames) in os.walk("Audioset_WAV_files/"+ largest + '/'):
    f.extend(filenames)
    break

shuffle(f)

clips_to_remove = f[:difference]
print(clips_to_remove)

for clip in clips_to_remove:
    os.remove("Audioset_WAV_files/"+ largest + '/' + clip)

# Check it has worked
coughs_count=0
for files in os.listdir("Audioset_WAV_files/Coughs/"):
    if files.endswith('.wav'):
        coughs_count+=1
print("Unedited number of cough clips:", coughs_count)


speech_count=0
for files in os.listdir("Audioset_WAV_files/Speech/"):
    if files.endswith('.wav'):
        speech_count+=1
print("Unedited number of speech clips:", speech_count)

Unedited number of cough clips: 265
Unedited number of speech clips: 272
['nIUbSRDTjAg.wav', 'i5p7QYtzdXA.wav', 'jQLV1M-dHuY.wav', 'eeG5d-BMKjA.wav', 'swGHDcYiesw.wav', '97d7nZH-nXI.wav', 'lUf8KCNHh0M.wav']
Unedited number of cough clips: 265
Unedited number of speech clips: 265


# Save YouTube IDs of final clip selections

In [13]:
# get filename of every file in both directories
# write to CSV  selected_YouTubeIDs

coughs_clips = ["Coughs"]
for files in os.listdir("Audioset_WAV_files/Coughs/"):
    if files.endswith('.wav'):
        coughs_clips.append(files)

speech_clips = ["Speech"]
for files in os.listdir("Audioset_WAV_files/Speech/"):
    if files.endswith('.wav'):
        speech_clips.append(files)

csvData = zip(coughs_clips, speech_clips)      
with open('selected_YouTubeIDs.csv', 'w') as csvFile:
    writer = csv.writer(csvFile)
    writer.writerows(csvData)